In [1]:
import pandas as pd

attribute_list = ["age", 
                  "workclass", 
                  "education", 
                  "years_of_education", 
                  "marital_status", 
                  "occupation", 
                  "relationship", 
                  "race", 
                  "sex", 
                  "hours_per_week", 
                  "native_country", 
                  "salary_bracket"]

measure_list = [
    "age",
    "years_of_education", 
    "capital_gain", 
    "capital_loss", 
    "hours_per_week"
]

sql_dict = {}
ref_dict = {}
query_dict = {}

def exec_sql_query(query, param=None):
    
    from sqlalchemy import create_engine
    import urllib
    params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};SERVER=LAPTOP-C3LFVOFI;DATABASE=student_life;UID=student_sense;PWD=abhinav123")
    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    connection = engine.raw_connection()
    
    try:
        cursor = connection.cursor()
        if(param):
            cursor.execute(query, param)
        else : 
            cursor.execute(query)
            
        results = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        df = pd.DataFrame.from_records(results, columns=columns)
        cursor.close()
        connection.commit()
    finally:
        connection.close()
    
    del engine
    return df


In [2]:
for attribute in attribute_list:
    for measure in measure_list:
        ref_sql = "select "+ attribute + ", avg(" + measure +") as avg, max("+ measure + ") as max, min("+ measure + ") as min, sum("+ measure + ") as sum, count("+ measure + ") as count from adult where marital_status in  ('Married-AF-spouse' , 'Married-spouse-absent', 'Married-civ-spouse')  group by "+ attribute
        query_sql = "select "+ attribute + ", avg(" + measure +") as avg, max("+ measure + ") as max, min("+ measure + ") as min, sum("+ measure + ") as sum, count("+ measure + ") as count from adult where marital_status not in  ('Married-AF-spouse' , 'Married-spouse-absent', 'Married-civ-spouse')  group by "+ attribute
#         sql_dict[(attribute, measure)] = sql
        ref_dict[(attribute, measure)] = exec_sql_query(ref_sql)
        query_dict[(attribute, measure)] = exec_sql_query(query_sql)
        
        



In [3]:
from IPython.display import display
from sklearn.preprocessing import normalize
from scipy.stats import entropy
import operator    

utility_score = {}
new_ref_dict = {}
new_query_dict = {}
 
for key in ref_dict:
    attribute, measure = key
    
    ref_view = ref_dict[key]
    query_view = query_dict[key]
    
    joined_df = ref_view.merge(right=query_view ,on=attribute, how='inner',  suffixes = ("_ref", "_query"), sort=True)
    ref_view = joined_df.iloc[:,0:6]
    query_view =  pd.concat([joined_df.iloc[:,0],joined_df.iloc[:,6:]], axis=1)
    
    new_ref_dict[(attribute, measure)] = ref_view
    new_query_dict[(attribute, measure)] = query_view
    
    function_list = ["avg", "max", "min", "sum", "count"]
    
    for i in range(1, 6):
        ref_measure = ref_view.iloc[:,i].as_matrix()
        query_measure = query_view.iloc[:,i].as_matrix()
        kl_divergence = entropy(ref_measure, query_measure)
        utility_score[(attribute, measure, function_list[i-1])] = kl_divergence
#         print("kl_divergence", kl_divergence)
    
print(utility_score)

# sorted_utility_score = sorted(utility_score.items(), key=operator.itemgetter(1), reverse=True)
# print(sorted_utility_score)   
# print(type(sorted_utility_score))
    
    
    

C:\Users\Abhinav Shaw\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2508: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=0)
C:\Users\Abhinav Shaw\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2515: RuntimeWarning: invalid value encountered in true_divide
  qk = 1.0*qk / np.sum(qk, axis=0)


{('age', 'age', 'avg'): 0.0, ('age', 'age', 'max'): 0.0, ('age', 'age', 'min'): 0.0, ('age', 'age', 'sum'): 0.20765029647544539, ('age', 'age', 'count'): 0.32782739381929366, ('age', 'years_of_education', 'avg'): 0.0076841173850884211, ('age', 'years_of_education', 'max'): 0.0071442173833017059, ('age', 'years_of_education', 'min'): 0.15115339140441469, ('age', 'years_of_education', 'sum'): 0.32305078402583531, ('age', 'years_of_education', 'count'): 0.32782739381929366, ('age', 'capital_gain', 'avg'): inf, ('age', 'capital_gain', 'max'): inf, ('age', 'capital_gain', 'min'): inf, ('age', 'capital_gain', 'sum'): inf, ('age', 'capital_gain', 'count'): 0.32782739381929366, ('age', 'capital_loss', 'avg'): inf, ('age', 'capital_loss', 'max'): inf, ('age', 'capital_loss', 'min'): inf, ('age', 'capital_loss', 'sum'): inf, ('age', 'capital_loss', 'count'): 0.32782739381929366, ('age', 'hours_per_week', 'avg'): 0.023898681071293143, ('age', 'hours_per_week', 'max'): 0.039777661795872554, ('age'

In [5]:
import pickle
       
with open('ref_data.pickle', 'wb') as handle:
    pickle.dump(new_ref_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('query_data.pickle', 'wb') as handle:
    pickle.dump(new_query_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('utility_score.pickle', 'wb') as handle:
    pickle.dump(utility_score, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:

    
with open('ref_data.pickle', 'rb') as handle:
    ref_data = pickle.load( handle)
    
with open('query_data.pickle', 'rb') as handle:
    query_data = pickle.load(handle)
    
with open('utility_score.pickle', 'rb') as handle:
    utility_score = pickle.load(handle)
    
    

In [7]:
print(utility_score)

{('age', 'age', 'avg'): 0.0, ('age', 'age', 'max'): 0.0, ('age', 'age', 'min'): 0.0, ('age', 'age', 'sum'): 0.20765029647544539, ('age', 'age', 'count'): 0.32782739381929366, ('age', 'years_of_education', 'avg'): 0.0076841173850884211, ('age', 'years_of_education', 'max'): 0.0071442173833017059, ('age', 'years_of_education', 'min'): 0.15115339140441469, ('age', 'years_of_education', 'sum'): 0.32305078402583531, ('age', 'years_of_education', 'count'): 0.32782739381929366, ('age', 'capital_gain', 'avg'): inf, ('age', 'capital_gain', 'max'): inf, ('age', 'capital_gain', 'min'): inf, ('age', 'capital_gain', 'sum'): inf, ('age', 'capital_gain', 'count'): 0.32782739381929366, ('age', 'capital_loss', 'avg'): inf, ('age', 'capital_loss', 'max'): inf, ('age', 'capital_loss', 'min'): inf, ('age', 'capital_loss', 'sum'): inf, ('age', 'capital_loss', 'count'): 0.32782739381929366, ('age', 'hours_per_week', 'avg'): 0.023898681071293143, ('age', 'hours_per_week', 'max'): 0.039777661795872554, ('age'

In [19]:
# display(ref_data[('age', 'age')])
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

C:\Users\Abhinav Shaw\Anaconda3\python.exe
3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)
